In [0]:
service_credential = dbutils.secrets.get(scope="azure-key-vault-secret1",key="datalake-secret")

spark.conf.set("fs.azure.account.auth.type.rkudatalakegen2.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.rkudatalakegen2.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.rkudatalakegen2.dfs.core.windows.net", "e1c699e2-256e-4508-a129-340636ed905f")
spark.conf.set("fs.azure.account.oauth2.client.secret.rkudatalakegen2.dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.rkudatalakegen2.dfs.core.windows.net", "https://login.microsoftonline.com/6dfaeccd-2d4f-43fc-8db8-8298cf6e47f1/oauth2/token")

In [0]:
table_name=[]

for n in dbutils.fs.ls("abfss://silver@rkudatalakegen2.dfs.core.windows.net/SalesLT/"):
    table_name.append(n.name[:-1])

['Address', 'Customer', 'CustomerAddress', 'Product', 'ProductCategory', 'ProductDescription', 'ProductModel', 'ProductModelProductDescription', 'SalesOrderDetail', 'SalesOrderHeader']


In [0]:
import pyspark
for each_table in table_name:
    input_path = f'abfss://silver@rkudatalakegen2.dfs.core.windows.net/SalesLT/{each_table}/'
    df = spark.read.format('delta').load(input_path)
    column_names = df.columns
    for col in column_names:
        new_col="".join([f"_{col[i]}" if col[i].isupper() and i!=0 and not col[i-1].isupper() else col[i] for i in range(len(col))])
        
        df = df.withColumnRenamed(col,new_col)
    output_path = f'abfss://gold@rkudatalakegen2.dfs.core.windows.net/SalesLT/{each_table}/'
    df.write.format('delta').mode("overwrite").save(output_path)